In [1]:
from __future__ import division
from collections import OrderedDict
from scipy import integrate
from math import pi
import numpy as np

from grbTools.cosmology.luminositydistance import Cosmology
from grbTools.cosmology.luminositydistance import LumDist     # This is a basic option.

In [ ]:
import pandas as pd
import numpy as np

f = "/Users/kimzoldak/GRBs/sample/Band_13_GBM+LAT__19_GBMconstrained.txt"

df = pd.read_csv(f)

df.head()

burst = 'bn080916009'

values = []
for par in 'alpha beta epeak norm'.split():
    values.append(float(df[df.trigger == burst][par].values))
    

values

In [ ]:
class Flux(object):
    
    def __init__(self, modelname, parameters, emin, emax, redshift=None, photonflux=False, program=None):
        self.keVtoerg = 1.60217657E-9
        self.modelname = modelname
        self.photonflux = photonflux
        
        if isinstance(parameters, dict) is False:
            raise Exception("parameters must be a dictionary type.")
        self.parameters = parameters

        if redshift is not None:
            self.redshift = redshift
            self.emin = emin/(1.+redshift)
            self.emax = emax/(1.+redshift)
        else:
            msg_z = ("\n *** WARNING: *** \n You are using observer-frame "
                    "fluxes. Eiso requires rest-frame fluxes. Please use "
                    "redshift if computing fluxes for Eiso. \n")
            print(msg_z)
            self.emin = emin
            self.emax = emax
        
        # clean up program name if defined.
        if program is not None:
            program = program.lower().replace(' ','')
        self.program = program
        
        if self.program is None:
            from general.models import lpow, copl, band, grbm, sbpl, bbody  # band and grbm are IDENTICAL!
            self.modelOptions = ["lpow", "copl", "band", "grbm", "sbpl", "bbody"]
            self._allfuncs = [lpow, copl, band, grbm, sbpl, bbody]
            
        elif self.program == 'rmfit':
            from rmfit.models import lpow, copl, band, sbpl, bbody
            self.modelOptions = ["lpow", "copl", "band", "sbpl", "bbody"]
            self._allfuncs = [lpow, copl, band, sbpl, bbody]
            
        elif (self.program == 'xspec') or (self.program == 'bxa'): 
            from xspec.modesl import lpow, powerlaw, cutoffpl, grbm, sbpl, bbody
            self.modelOptions = ["lpow", "powerlaw", "cutoffpl", "grbm", "sbpl", "bbody"]
            self._allfuncs = [lpow, powerlaw, cutoffpl, grbm, sbpl, bbody]
            
        else:
            programOptions = [None, 'rmfit', 'xspec', 'bxa']
            raise Exception("'program' unknown. Must be one of following: %r"%programOptions)
        
        funcs = []
        # Return only the necessary model functions, in order as needed.    
        for mod in self.modelname.split('+'):
            if mod not in self.modelOptions:
                raise Exception('Model component not known. %s'%mod)
            func = eval('lambda x,values: x * %s(x, *values)'%(mod))
            funcs.append(func)
        return funcs
            
                
    def calc_flux(self): 
        #funcs = self._get_flux_model_functions()  # list of only the ones you need and in the correct order!
        self.funcs = self._get_flux_model_functions()  
        totalFlux = []
        for mod,func in zip(self.modelname.split('+'), self.funcs):
            pars = self.parameters[mod].values()
            #func = eval('lambda x,values: x * %s(x, *values)'%(modpart))
            flux = integrate.quad(func, self.emin, self.emax, args=(pars), limit=100)[0]
            if self.photonflux is True:
                totalFlux.append(flux)
            else:
                nrgFlux = flux*self.keVtoerg
                totalFlux.append(nrgFlux)
        return np.sum(totalFlux)


In [35]:
parsDict = {'grbm':{'alpha':-1.0295860000000001, 'beta':-2.198223, 'epeak':510.64, 'norm':0.017522454}}

In [36]:
kim = Energetics(modelname='grbm', 
                  parameters=parsDict, 
                  emin=10.0, 
                  emax=10000.0, 
                  redshift=4.35, 
                  photonflux=False, 
                  program=None)

In [37]:
kim.calc_flux()

NameError: name 'grbm' is not defined

In [ ]:
class Energetics(object):
    def __init__(self, modelname, parameters, emin, emax, redshift=None, photonflux=False, program=None):
        self.modelname = modelname
        self.photonflux = photonflux
        if isinstance(parameters, dict) is False:
            raise Exception("parameters must be a dictionary type.")
        self.parameters = parameters # should be a dict.
        if redshift is not None:
            self.redshift = redshift
            self.emin = emin/(1.+redshift)
            self.emax = emax/(1.+redshift)
        else:
            msg_z = ("\n *** WARNING: *** \n You are using observer-frame "
                    "fluxes. Eiso requires rest-frame fluxes. Please use "
                    "redshift if computing fluxes for Eiso. \n")
            print(msg_z)
            self.emin = emin
            self.emax = emax
        self.keVtoerg = 1.60217657E-9

        if program is not None:
            program = program.lower().replace(' ','')
            self.program = program
            if (program == 'xspec') or (program == 'bxa'):
                # They use identical functions, however prop of uncertainty is different. 
                raise Exception('Program not set up yet.')
            elif program == 'rmfit':
                raise Exception('Program not set up yet.')
            else:
                raise Exception("Program must be 'xspec', 'bxa', 'rmfit' or None ")
        else:
            self.program = None
            # Otherwise, use general model functions.
            model_options = ["lpow", "copl", "band", "sbpl", "bbody"]
            if self.modelname not in model_options:
                msg1 = "'model' must be one of following: %r"%model_options
                raise Exception(msg1)
            #from general.models import band # ***************************************** CHANGE THIS!!
            from general.models import eval('%s'%self.modelname)
            #self.func = band
                


    def calc_flux_general(self, uncertainty=False):
        if self.program is None and uncertainty is True:
            msg = ('There is no way to calculate uncertainty without specifying'
                    ' a program.')
            raise Exception(msg)
        unc = uncertainty # True or False
        totalFlux = []
        for modpart in self.modelname.split('+'):
            pars = self.parameters[modpart].values()
            func = eval('lambda x,values: x * %s(x, *values)'%(modpart))
            Flux = integrate.quad(func, self.emin, self.emax, args=(pars), limit=100)[0]
            if self.photonflux is True:
                totalFlux.append(Flux)
            else:
                nrgFlux = Flux*self.keVtoerg
                totalFlux.append(nrgFlux)
        return np.sum(totalFlux)

In [ ]:
# band(energy, alpha, beta, enterm, norm, entype='epeak', enorm=100.0)
paramDict = dict({'band':{'alpha':-1.2, 'beta': -2.8, 'enterm':502., 'norm':0.01, 'entype':'epeak'}})


energetics = Energetics(modelname='band', parameters=paramDict, 
                        emin=10., emax=10000., 
                        redshift=None, photonflux=False, program=None)

In [ ]:
    def calc_flux_xspec(self, uncertainty=False):
        if self.program is None and uncertainty is True:
            msg = ('There is no way to calculate uncertainty without specifying'
                    ' a program.')
            raise Exception(msg)
            unc = uncertainty # True or False
        pass
    
    def calc_flux_rmfit(self, uncertainty=False):
        if self.program is None and uncertainty is True:
            msg = ('There is no way to calculate uncertainty without specifying'
                    ' a program.')
            raise Exception(msg)
            unc = uncertainty # True or False
        pass


    def calc_fluence(self, uncertainty=False):
        if self.program is None and uncertainty is True:
            msg = ('There is no way to calculate uncertainty without specifying'
                    ' a program.')
            raise Exception(msg)
            unc = uncertainty # True or False
        pass

    def calc_eiso(self, redshift, duration, lumdistbasic=True, uncertainty=False):
        if self.program is None and uncertainty is True:
            msg = ('There is no way to calculate uncertainty without specifying'
                    ' a program.')
            raise Exception(msg)
            unc = uncertainty # True or False

            if lumdistbasic is True:
                DL = LumDist(self.redshift, cosmoconstants=None) # We want defaults.
            else:
                cosmo = Cosmology()
                cosmo.luminosity_distance(self.redshift, 
                                            model='FlatLambdaCDM', 
                                            units='cm', 
                                            **constants)
            luminosity_distance(self, redshifts, model='FlatLambdaCDM', 
                            units=None, **constants)
            eiso = (4.0 * pi * DL**2 * calc_fluence(unc))/(1.+self.redshift)
        return eiso

In [ ]:
# band(energy, alpha, beta, enterm, norm, entype='epeak', enorm=100.0)
paramDict = dict({'band':{'alpha':-1.2, 'beta': -2.8, 'enterm':502., 'norm':0.01, 'entype':'epeak'}})


energetics = Energetics(modelname='band', parameters=paramDict, 
                        emin=10., emax=10000., 
                        redshift=None, photonflux=False, program=None)

In [ ]:
energetics.calc_flux_general()